In [1]:
from qiskit import IBMQ

# Load Account and Choose Backend
IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q')
name = 'ibmqx2'
backend = provider.get_backend(name) # ibmq_16_melbourne, ibmq_vigo, ibmq_london, ibmq_burlington, ibmq_essex, ibmqx2, ibmq_ourense

In [2]:
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.insert(1, '../Scripts')
from measfilter import param_record, getData0,read_params
from gatesimu import Gateexp, output_gate, read_data, gate_circ

# Parameters
interested_qubits = [1,2]
gate_num = 200
gate_type = 'X'

itr = 16
shots = 8192
file_address = 'T1/'
noise_file_address = 'Noise/'

# Collect U3-Gate Data
param_record(backend, itr = itr, shots = shots, file_address = noise_file_address)
for q in interested_qubits:
    Gateexp(gate_num, gate_type, q, itr, backend, file_address =file_address)

ModuleNotFoundError: No module named 'measfilter'

In [ ]:
import csv
prop_dict = backend.properties().to_dict()
with open(noise_file_address + 'given_params.csv', mode='w', newline='') as sgm:
    param_writer = csv.writer(sgm, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for q in interested_qubits:
        p0m0 = 1 - prop_dict['qubits'][q][5]['value']
        if p0m0 == 1 or p0m0 < 0.7:
            p0m0 = 0.9
        p1m1 = 1 - prop_dict['qubits'][q][4]['value']
        if p1m1 == 1 or p1m1 < 0.7:
            p1m1 = 0.9
        param_writer.writerow([p0m0,p1m1,prop_dict['gates'][4*q+3]['parameters'][0]['value']])

In [ ]:
import seaborn as sns
import pandas as pd

M = 40000
gate_sd = 0.01
meas_sd = 0.1
info = {}

params = read_params(file_address = noise_file_address)
for q in interested_qubits:
    print('Qubit ',q)
    data = read_data(q, gate_type, gate_num, file_address = file_address)
    d = getData0(data, int(itr*shots/1024), q)
    _,post_lambdas = output_gate(d, q, M, params, gate_sd, meas_sd, gate_type, gate_num, file_address = file_address)
    info['Qubit' + str(q)] = post_lambdas

In [ ]:
from gatesimu import plotComparsion
for q in interested_qubits:
    d = getData0(read_data(q, gate_type, gate_num, file_address = file_address), int(itr*shots/1024), q)
    plotComparsion(d, info['Qubit'+str(q)], q, file_address = file_address)

In [ ]:
import seaborn as sns
qubit_num = 1
our_q1 = pd.DataFrame(info['Qubit' + str(qubit_num)],columns=['P0','P1','eps'])
sns_plot = sns.jointplot(x="P0", y="P1", data=our_q1, kind="kde", xlim=(0.6,1.02), ylim=(0.6,1.02)).set_axis_labels('$1 - \epsilon_{m0,1}$', '$1 - \epsilon_{m1,1}$')
sns_plot.fig.savefig(noise_file_address + 'p0p1-Qubit{}.jpg'.format(qubit_num)) 

In [ ]:
import numpy as np
from measfilter import closest_average, closest_mode
q = 1
postSB = pd.read_csv(file_address + 'StandPostQubit{}.csv'.format(q)).to_numpy()
print(np.mean(postSB,axis = 0))
print(closest_average(postSB))
print(closest_mode(postSB))

In [ ]:
q = 2
postSB = pd.read_csv(file_address + 'StandPostQubit{}.csv'.format(q)).to_numpy()
print(np.mean(postSB,axis = 0))
print(closest_average(postSB))
print(closest_mode(postSB))